In [6]:
from pupil_apriltags import Detector, Detection
from collections import defaultdict
from typing import List, Dict, Tuple, Any
from glob import glob
import numpy as np
import os
import time
import sys
import cv2
from math import atan2, degrees

In [2]:
#manual_detection.py
#ignore for use on local machine, unfeasible to run without overclocking, even so, takes unreasonable amount of time


def extract_frames(video_path: str, frames_path: str) -> None:
    """Convert a video (mp4 or similar) into a series of individual PNG frames.
    Make sure to create a directory to store the frames before running this function.
    Args:
        video_path (str): filepath to the video being converted
        frames_path (str): filepath to the target directory that will contain the extract frames
    """
    video = cv2.VideoCapture(video_path)
    count = 0
    success = 1

    # Basically just using OpenCV's tools
    while success:
        success, frame = video.read()
        cv2.imwrite(f'{frames_path}/frame{count}.png', frame)
        count += 1

    # Optional print statement
    print(f'Extracted {count} frames from {video_path}.')


def detect_tags(frames_path: str, aperture=11, visualize=False) -> Tuple[List[List[Dict[str, Any]]], Dict[int, int]]:
    """Detect all tags (Apriltags3) found in a folder of PNG files and return (1) a list of tag objects
    for preprocessing and (2) a dictionary containing the frequency that each tag ID appeared
    Args:n
        frames_path (str): path to the directory containing PNG images
        aperture (int):
        visualize (bool):
    Returns:
        frames (List[Dict[str, Any]]): list of objects containing id (int), centroid (np.array[int]) and corners (np.array[int])
        tag_ds (Dict[int, int]): dictionary mapping tag IDs to frequency of tag across all images
    """
    # Initialize variables
    frames = []
    tag_ids = defaultdict(int)
    at_detector = Detector()

    # Sort by index in.../frame<index>.png
    all_images = sorted(glob(f'{frames_path}/*.png'), key=lambda f: int(os.path.basename(f)[5:-4]))

    # Deleted last image after out of range error popped up
    # TODO: but not analyzing last 2 frames?
    # Feb 21: commented out deleting last frame, altho this was important when you have all frames in folder
    print(len(all_images))
    if len(all_images) > 1:
        all_images = all_images[:-1]

    num_images = len(all_images)
    #print_progress_bar(0, num_images, prefix='Progress:', suffix='Complete', length=50)

    # Iterate thru all PNG images in frames_path
    for i, img_path in enumerate(all_images):
        # Create a grayscale 2D NumPy array for Detector.detect()
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if type(img) == np.ndarray:
            tags_in_framex = []
            for tag in at_detector.detect(img):
                # Increment frequency
                tag_ids[tag.tag_id] += 1
                # r = np.roll(np.float32(img), tag.homography + 1, axis=0)
                # Add to frames in following format - feel free to adjust
                tags_in_framex.append({
                    'id': tag.tag_id,
                    'id_confidence': tag.decision_margin,
                    'soft_id': tag.tag_id,
                    'perimeter': 100, #cv2.arcLength(img, closed=True),
                    'centroid': tag.center,
                    'verts': tag.corners,
                    'frames_since_true_detection': 0
                })

                # {'id': msg, 'id_confidence': id_confidence, 'verts': r.tolist(), 'soft_id': soft_msg,
                #  'perimeter': cv2.arcLength(r, closed=True), 'centroid': centroid.tolist(),
                #  "frames_since_true_detection": 0}
            frames.append(tags_in_framex)
        time.sleep(0.01)
        print_progress_bar(i + 1, num_images, prefix='Progress:', suffix='Complete', length=50)

    return frames, dict(tag_ids)


def print_progress_bar (iteration, total, prefix ='', suffix ='', decimals = 1, length = 100, fill ='█', printEnd ="\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    sys.stdout.write('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix))
    # Print New Line on Complete
    if iteration == total:
        print()


def main():
    # Define path folder
    # test path 1 from 10/7 simulator recording - not all tags are id b/c lighting
    # path = "/home/whitney/Teresa/demos/surfaceTestUpdatedTags/001"
    # test path 2 from 10/7 simulator recording - not all tags are id b/c lighting
    # path = "/home/whitney/Teresa/demos/surfaceTestUpdatedTags/002"
    # test path 3 from prev recording at desk with reflection - all tags id success
    # path = '/home/whitney/Teresa/demos/testingSurfaces_Teresa'
    # test path to id tags
    # path = '/home/whitney/Teresa/demos/idtags/013'
    # test path 4 from 10/8 recording at lab with good lighting
    # path = '/home/whitney/Teresa/demos/surfaceTestUpdatedTagsLab/014'
    # test path from 10/15 recording at lab with eye & good lighting
    # path = '/home/whitney/Teresa/demos/surfaceTestLabEye/000'
    # test path from 10/17 recording at lab with calibration
    # path = '/home/whitney/Teresa/demos/surfaceANDcalibration_3screens'
    # test path from 10/25 recording
    #path = '/media/whitney/New Volume/Teresa/SD_grant_EM/Eye_Recordings/Subject1/001'
    path = 'C:/Users/hbass/Desktop/SP2020 Research/Subject1/001'
    # test path from 11/4 recording
    #path = '/home/whitney/Teresa/demos/2019_11_04/000'
    # id tag
    # path = '/home/whitney/recordings/2019_11_01/014'

    # Create video path
    video_path = path + "/world.mp4"
    # Create frame path using OS package
    # Define the name of the directory to be created
    frames_path = path + "/frames"
    try:
        if not os.path.exists(frames_path):
            os.mkdir(frames_path)
        else:
            print("Successfully created the directory %s " % frames_path)
    except OSError:
        print("Creation of the directory %s failed" % frames_path)
    # Detect tags in frames
    extract_frames(video_path, frames_path)
    frames, tag_ids = detect_tags(frames_path)

    # Descriptive print statements
    tag_count = sum(count for count in tag_ids.values())
    print(f'Detected {tag_count} tags in {len(frames)} frames.')
    print(f'Found IDs of {list(tag_ids.keys())}.')


if __name__ == '__main__':
    main()

Creation of the directory C:/Users/hbass/Desktop/SP2020 Research/Subject1/001/frames failed
Extracted 1 frames from C:/Users/hbass/Desktop/SP2020 Research/Subject1/001/world.mp4.
0
Detected 0 tags in 0 frames.
Found IDs of [].


In [3]:
def detect_tags_test(frames_path: str, aperture=11, visualize=False) -> Tuple[List[List[Dict[str, Any]]], Dict[int, int]]:
    """Detect all tags (Apriltags3) found in a folder of PNG files and return (1) a list of tag objects
    for preprocessing and (2) a dictionary containing the frequency that each tag ID appeared
    Args:n
        frames_path (str): path to the directory containing PNG images
        aperture (int):
        visualize (bool):
    Returns:
        frames (List[Dict[str, Any]]): list of objects containing id (int), centroid (np.array[int]) and corners (np.array[int])
        tag_ds (Dict[int, int]): dictionary mapping tag IDs to frequency of tag across all images
    """
    # Initialize variables
    frames = []
    tag_ids = defaultdict(int)
    at_detector = Detector()

    # Sort by index in.../frame<index>.png
    all_images = sorted(glob(f'{frames_path}/*.png'), key=lambda f: int(os.path.basename(f)[5:-4]))

    # Deleted last image after out of range error popped up
    # TODO: but not analyzing last 2 frames?
    # Feb 21: commented out deleting last frame, altho this was important when you have all frames in folder
    print(len(all_images))
    if len(all_images) > 1:
        all_images = all_images[:-1]

    num_images = len(all_images)
    print_progress_bar(0, num_images, prefix='Progress:', suffix='Complete', length=50)

    # Iterate thru all PNG images in frames_path
    for i, img_path in enumerate(all_images):
        # Create a grayscale 2D NumPy array for Detector.detect()
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if type(img) == np.ndarray:
            tags_in_framex = []
            for tag in at_detector.detect(img):
                # Increment frequency
                tag_ids[tag.tag_id] += 1
                # r = np.roll(np.float32(img), tag.homography + 1, axis=0)
                # Add to frames in following format - feel free to adjust
                tags_in_framex.append({
                    'id': tag.tag_id,
                    'id_confidence': tag.decision_margin,
                    'soft_id': tag.tag_id,
                    'perimeter': 100, #cv2.arcLength(img, closed=True),
                    'centroid': tag.center,
                    'verts': tag.corners,
                    'frames_since_true_detection': 0
                })

                # {'id': msg, 'id_confidence': id_confidence, 'verts': r.tolist(), 'soft_id': soft_msg,
                #  'perimeter': cv2.arcLength(r, closed=True), 'centroid': centroid.tolist(),
                #  "frames_since_true_detection": 0}
            frames.append(tags_in_framex)
        time.sleep(0.01)
        print_progress_bar(i + 1, num_images, prefix='Progress:', suffix='Complete', length=50)

    return frames, dict(tag_ids)

# Beginning my experimentation
## Comments on each block for what it is used for. Do not run cells with ignore tag
## Will clean and scale better after initial image proves feasible

In [4]:
#returns the output of the detect tags on the specific image 4743
frame, tag_ids = detect_tags_test('C:/Users/hbass/Desktop/SP2020 Research/Subject1/Subject1/001/frames_selected')
print(frame[0])

1
Progress: |██████████████████████████████████████████████████| 100.0% Complete
[{'id': 0, 'id_confidence': 99.00302124023438, 'soft_id': 0, 'perimeter': 100, 'centroid': array([355.7077122 , 220.49514557]), 'verts': array([[342.56369019, 236.55683899],
       [369.70407104, 234.2318573 ],
       [369.14724731, 204.07234192],
       [341.66320801, 206.71118164]]), 'frames_since_true_detection': 0}, {'id': 1, 'id_confidence': 96.2584457397461, 'soft_id': 1, 'perimeter': 100, 'centroid': array([362.15848359, 351.72380703]), 'verts': array([[349.66488647, 366.97045898],
       [376.85266113, 366.11090088],
       [374.76672363, 336.33724976],
       [347.39718628, 337.27099609]]), 'frames_since_true_detection': 0}, {'id': 2, 'id_confidence': 85.86864471435547, 'soft_id': 2, 'perimeter': 100, 'centroid': array([374.80622407, 477.69332171]), 'verts': array([[363.44937134, 491.2350769 ],
       [389.72055054, 491.75695801],
       [386.41009521, 463.85702515],
       [359.70401001, 463.4525

In [5]:
#function that gets what attribute is of the dictionary generated by detect tags a user needs
#for instance, I needed the centers of each tag and the corners to crop the image, so this function acquires them from the
#return call of detect tags
def attribute(frame, feature):
    qr_codes = frame[0]
    attributes = []
    for i in range(len(qr_codes)):
        qr_code = qr_codes[i]
        print(feature + ' of tag id ' + str(i) + ":", qr_code[feature])
        attributes.append(qr_code[feature])
    return attributes
    

In [6]:
#attains the centers of each QR code of the frame; not sorted in any order
centers = attribute(frame, 'centroid')
#comment for myself: from corners, iterate through and find the corners of highest top left, highest top right, lowest bottom left, lowest bottom right, and then draw a box around the image

centroid of tag id 0: [355.7077122  220.49514557]
centroid of tag id 1: [362.15848359 351.72380703]
centroid of tag id 2: [374.80622407 477.69332171]
centroid of tag id 3: [514.78063242 479.03755271]
centroid of tag id 4: [665.59619704 477.22884689]
centroid of tag id 5: [812.30464889 472.72394708]
centroid of tag id 6: [817.15998013 343.63203354]
centroid of tag id 7: [816.99070099 209.07865746]
centroid of tag id 8: [662.2222816  206.60730277]
centroid of tag id 9: [502.67909167 210.40346754]


In [7]:
#attains the corners for each of the QR codes in the frame; not sorted in any order
corners = attribute(frame, 'verts')

verts of tag id 0: [[342.56369019 236.55683899]
 [369.70407104 234.2318573 ]
 [369.14724731 204.07234192]
 [341.66320801 206.71118164]]
verts of tag id 1: [[349.66488647 366.97045898]
 [376.85266113 366.11090088]
 [374.76672363 336.33724976]
 [347.39718628 337.27099609]]
verts of tag id 2: [[363.44937134 491.2350769 ]
 [389.72055054 491.75695801]
 [386.41009521 463.85702515]
 [359.70401001 463.45251465]]
verts of tag id 3: [[500.83267212 493.3616333 ]
 [530.77661133 493.43069458]
 [528.99859619 464.43618774]
 [498.6008606  464.47903442]]
verts of tag id 4: [[650.16571045 492.15579224]
 [681.10949707 491.16586304]
 [680.94244385 462.38339233]
 [649.87756348 463.10736084]]
verts of tag id 5: [[797.19696045 487.66009521]
 [825.66931152 486.29040527]
 [827.50085449 457.70028687]
 [798.7355957  458.95001221]]
verts of tag id 6: [[802.15020752 359.21655273]
 [831.45959473 358.93811035]
 [832.23144531 327.98345947]
 [802.72357178 328.17953491]]
verts of tag id 7: [[802.28314209 224.22209167]


In [ ]:
#Not needed anymore since we use a bitwise masking procedure to crop the image. This was done using the
#cv2 rectangle function which requires an upper left and bottom right corner. Keeping for future reference on how to
#use the cv2 rectangle function.

#TESTING FOR OPTIMAL POINT: drawing of the rectangle using cv2, need an upper left and bottom right corner
for i in range(len(corners)):
    sq = corners[i]
    for j in range(len(sq)):
        coord = sq[j]
        path = 'C:/Users/hbass/Desktop/SP2020 Research/Subject1/Subject1/001/frames_selected/frame7473.png'
        image = cv2.imread(path)
        window_name = 'Box'
        start_point= (341,206)
        end_point = (int(coord[0]), int(coord[1]))
        print(end_point)
        color = (255,0,0)
        thickness = 2
        image = cv2.rectangle(image, start_point, end_point, color, thickness)
        cv2.imshow(window_name, image);
        cv2.waitKey(0);

In [ ]:
#Not needed given bitwise mask below but good reference.
#The drawing of the image generated from the best coordinates from the cell above
#DRAWING HERE
path = 'C:/Users/hbass/Desktop/SP2020 Research/Subject1/Subject1/001/frames_selected/frame7473.png'
image = cv2.imread(path)
window_name = 'Box'
start_point= (341,206)
end_point = (825, 486)
color = (255,0,0)
thickness = 2
image = cv2.rectangle(image, start_point, end_point, color, thickness)
cv2.imshow(window_name, image);
cv2.waitKey(0);

In [ ]:
#Goes through all centroids of the QR codes and generates the lines create the XY grid through the center of the screen.
#When running, hit a button on the keyboard to filter through all combinations until an appropriate one is found.
#Prints out the coordinates that were used.

#Testing lines for appropriate XY grid 
#centroid for xy grid
for centroid1 in centers:
    for centroid2 in centers:
        path = 'C:/Users/hbass/Desktop/SP2020 Research/Subject1/Subject1/001/frames_selected/frame7473.png'
        image = cv2.imread(path)
        window_name = 'Box'
        start_point= (int(centroid1[0]),int(centroid1[1]))
        end_point = (int(centroid2[0]),int(centroid2[1]))
        print(start_point, end_point)
        color = (255,0,0)
        thickness = 2
        image = cv2.line(image, start_point, end_point, color, thickness)
        cv2.imshow(window_name, image);
        cv2.waitKey(0);
        

In [ ]:
#Bitwise masking that crops the image based on just what we want to see on the screen. Utilzes the corner
#coordiates from previous cell computations, and then filters through to see which is the best mask.
#Hit a keyboard button to filter through images.

#Testing for the cropping of the mask image 
for i in range(len(corners)):
    sq = corners[i]
    for j in range(len(sq)):
        coord = sq[j]
        x = int(coord[0])
        y = int(coord[1])
        print(x,y)
        im = cv2.imread('C:/Users/hbass/Desktop/SP2020 Research/Subject1/Subject1/001/frames_selected/frame7473.png')
        mask = np.zeros(im.shape, dtype=np.uint8)
        roi_corners = np.array([[(341, 206), (x,y), (831, 193), (825,486)]], dtype=np.int32)
        channel_count = im.shape[2]
        ignore_mask_color = (255,)*channel_count
        cv2.fillPoly(mask, roi_corners, ignore_mask_color)
        masked_image = cv2.bitwise_and(im, mask)
        cv2.imshow('masked', masked_image);
        cv2.waitKey(0);

In [ ]:
#Since the ROI_CORNERS parameter of the masking requires a specific ordering to crop the image, we need to find
#the best permutation of the coordinates calculated from before to generate the image we need
#Hit a button on the keyboard to filter through the images.
#Prints out coordinate.

#finding the appropriate permutation of the pixels that crop the screen
points = [(341, 206), (363, 491), (831, 193), (825,486)]
for i in range(len(points)):
    for j in range(len(points)):
        for k in range(len(points)):
            for n in range(len(points)):
                im = cv2.imread('C:/Users/hbass/Desktop/SP2020 Research/Subject1/Subject1/001/frames_selected/frame7473.png')
                mask = np.zeros(im.shape, dtype=np.uint8)
                print(points[i], points[j], points[k], points[n])
                roi_corners = np.array([[points[i], points[j], points[k], points[n]]], dtype=np.int32)
                channel_count = im.shape[2]
                ignore_mask_color = (255,)*channel_count
                cv2.fillPoly(mask, roi_corners, ignore_mask_color)
                masked_image = cv2.bitwise_and(im, mask)
                cv2.imshow('masked', masked_image);
                cv2.waitKey(0);

In [8]:
#returns the midpoint between two pixel points, and rounds them to nearest integer for use in drawing programs cv2line/circle
def midpoint(point1, point2):
    return (int((point1[0] + point2[0]) / 2) , int((point1[1] + point2[1]) / 2))

In [9]:
#run this cell to generate the XY grid on the specific image 
#optimal points found/masking the image to crop the screen
im = cv2.imread('C:/Users/hbass/Desktop/SP2020 Research/Subject1/Subject1/001/frames_selected/frame7473.png')
mask = np.zeros(im.shape, dtype=np.uint8)
roi_corners = np.array([[(341,206), (831,193), (825,486), (363,491)]], dtype=np.int32)
channel_count = im.shape[2]
ignore_mask_color = (255,)*channel_count
cv2.fillPoly(mask, roi_corners, ignore_mask_color)
masked_image = cv2.bitwise_and(im, mask)

#corners of the rectangle around the blob
upper_left = (600,375)
upper_right = (672, 375)
bottom_right = (672, 439)
bottom_left = (600, 439)

#drawing code to draw on the cropped image
start_point = (362 ,352)
end_point = (817, 343)
colour = (0,255,0)
thickness = 2
#X-AXIS
masked_image = cv2.line(masked_image, start_point, end_point, colour, thickness)
#Y-AXIS 
top_vert_line = midpoint((666,477), (515, 479))
bot_vert_line = midpoint((662,207), (503,210))
cv2.line(masked_image, top_vert_line, bot_vert_line , colour, thickness)
#ORIGIN
cv2.circle(masked_image, (586, 348), 2, (0,0,255), thickness)
#Testing to box around the blob
cv2.rectangle(masked_image, (600, 375), (672, 439), (255,0,0), thickness)

#center of the blob
cv2.line(masked_image, midpoint(upper_left, bottom_left), midpoint(upper_right, bottom_right), (0,0,255), thickness)
cv2.line(masked_image, midpoint(upper_left, upper_right), midpoint(bottom_left, bottom_right), (0,0,255), thickness)
cv2.circle(masked_image, (636,407), 2, colour, thickness)
cv2.imshow('XY_PLANE', masked_image);
cv2.waitKey(1000);
#cv2.imshow('mask',masked_image);
#cv2.imsave('C:/Users/hbass/Desktop/SP2020 Research/Subject1/Subject1/001/frames_selected/masked_image.jpg', masked_image)
#cv2.waitKey(0);


In [ ]:
#testing cell to find best fit origin
start_point = (362 ,352)
end_point = (817, 343)
top_vert_line = midpoint((666,477), (515, 479))
bot_vert_line = midpoint((662,207), (503,210))
origin_y = midpoint(top_vert_line, bot_vert_line)
origin_x = midpoint(start_point, end_point)
#found by tuning the values computed above
origin = (586,348)
print(origin)

# Using pixel convertor

In [7]:
#pixel to degree
#TODO: figure out what size_in_px represents
h = 19.65
d = 57.3
r = 1600
size_in_px = 72
deg_per_px = degrees(atan2(.5*h, d)) / (.5*r)
size_in_deg = size_in_px * deg_per_px
print('The size of the stimulus is %s pixels and %s visual degrees' \
    % (size_in_px, size_in_deg))

The size of the stimulus is 72 pixels and 0.8756694059682902 visual degrees


In [10]:
h = 19.65
d = 57.3
r = 1600
#TODO: figure out what size_in_deg represents for experiment
size_in_deg = 
deg_per_px = degrees(atan2(.5*h, d)) / (.5*r)
size_in_px = size_in_deg / deg_per_px
print ('The size of the stimulus is %s pixels and %s visual degrees' \
    % (size_in_px, size_in_deg))

The size of the stimulus is 328.89124370119777 pixels and 4 visual degrees
